In [ ]:
import os
import pickle
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import sklearn.metrics as metrics
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from keras import models

In [ ]:
%load_ext tensorboard

In [ ]:
rm -rf ./logs/

In [ ]:
!pip install imgaug==0.4.0
!pip install imgo

In [ ]:
from imgo import uptools

In [ ]:
class_names = ['amethyst_deceiver',
               'bolete',
               'chanterelle',
               'chicken_of_the_woods',
               'death_cap',
               'false_chanterelle',
               'false_morel',
               'fibrecap',
               'field_mushroom',
               'fly_agaric',
               'giant_puffball',
               'grey_oyster',
               'morel',
               'orange_peel',
               'roundhead',
               'saddle',
               'shaggy_inkcap',
               'stinkhorn',
               'waxcap',
               'yellow_stainer']


In [ ]:
%%time
ds = uptools.Image_Dataset("/floyd/input/ds_1", "h5", 256, normalize=True, manual_classes=class_names)

In [ ]:
ds.details(plot=True)

In [ ]:
def cnn():

    inputs = keras.Input(shape=(256,256,3))
    x = layers.Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(256,(3,3),activation='relu',padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(512,(3,3),activation='relu',padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    outputs = layers.Dense(20, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
n_epochs = 15
s_batch = 32

In [ ]:
%%time
ds_net_01 = cnn()
model_path = "ds_net_01.h5"


cp = [ModelCheckpoint(filepath=model_path,
                      monitor="val_accuracy",
                      verbose=1,
                      save_best_only=True)]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tc = [tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)]

history = ds_net_01.fit(ds.X_train,
                        ds.y_train,
                        validation_data=(ds.X_val,ds.y_val),
                        epochs=n_epochs,
                        batch_size=s_batch,
                        callbacks=[cp,tc])

with open(f"ds_net_01_history", "wb") as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
model = models.load_model(model_path)
(val_loss, val_acc) = model.evaluate(ds.X_val,ds.y_val,verbose=1)
(test_loss, test_acc) = model.evaluate(ds.X_test,ds.y_test,verbose=1)
print('---')
print(f'Val Loss: {val_loss}')
print(f'Val Accuracy: {val_acc}')
print('---')
print(f'Val Loss: {test_loss}')
print(f'Val Accuracy: {test_acc}')

In [ ]:
test_preds = np.argmax(model.predict(ds.X_test,verbose=1),axis=-1)
test_results_df = pd.DataFrame(test_preds,columns=['predicted'])
test_results_df['actual'] = pd.DataFrame(ds.y_test).idxmax(axis=1)
cm = metrics.confusion_matrix(test_results_df['actual'],test_results_df['predicted'])
cmn = np.true_divide(cm, cm.sum(axis=1, keepdims=True))*100
df_cmn = pd.DataFrame(cmn, class_names, class_names)

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(10,10))
sns.heatmap(df_cmn, cmap='Blues', cbar=False, annot=True,annot_kws={"size":11}, fmt='.0f', linewidth=0.5)
for t in ax1.texts:
    t.set_text(t.get_text()+'%')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix', fontdict={'fontsize':15})
plt.tight_layout()
plt.savefig("cm.png")
plt.show()